# Scraping User Activity

I'm trying to identify who the active users are on the market. Going to scrape the number of jobs that they have quouted in the last 30 days and use that as a measure of their activity

# Importing Libraries

In [1]:
# Packages for Scraping
from bs4 import BeautifulSoup
import requests
import random

# Packages for cleaning Data
import re
import pandas as pd
import numpy as np
import time

# Packages for PostgreSQL Import
# from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
# import psycopg2
import os

# Other
import itertools

# Defining Functions

In [2]:
def html_extract():
    """
    Extracts and cleans the html from a website (x)
    
    Returns soup object from beautiful soup
    """
    source =  requests.get(html).text
    soup = BeautifulSoup(source, 'html.parser')
    
    return soup

In [12]:
def extract_30day_activity():
    """
    Visits a users profile and extracts their activity in the last 30 days.
    """
    activity_html = soup_html.find('ul',class_="dropdown-menu module_list tighter rhythmPadding1").find_all('li')
    activity_text = [x.get_text() for x in activity_html]
    activity_clean = [x.replace('\n','') for x in activity_text]
    activity_30days_text = activity_clean[2].split('\r')
    activity_30days_text = [x.strip() for x in activity_30days_text]
    activity_30days = int(activity_30days_text[1])
    
    return activity_30days

In [13]:
def extract_30day_activity2():
    """
    Visits a users profile and extracts their activity in the last 30 days.
    """
    activity_html = soup_html.find('dl',class_="profile-attd").find_all('dd')
#     activity_text = [x.get_text() for x in activity_html]
#     activity_clean = [x.replace('\n','') for x in activity_text]
#     activity_30days_text = activity_clean[2].split('\r')
#     activity_30days_text = [x.strip() for x in activity_30days_text]
#     activity_30days = int(activity_30days_text[1])
    
    return activity_30days

# Looping over all user htmls (I have scraped)

## Importing Data

In [4]:
# Names for accessing SQL database
dbname = "freelance_db"
username = os.environ['USER']
pswd = os.environ['SQLPSWD']

# Connect to SQL Database
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# Importing Data
sql_query = """SELECT profile_url FROM analysis_dummies_table;"""
dt = pd.read_sql_query(sql_query, con)

## Extracting activity in last 30 days

In [ ]:
soup = html_extract('https://www.guru.com/freelancers/scopic')
extract_stats = 

In [9]:
urls = ['https://www.guru.com/freelancers/scopic', 'https://www.guru.com/freelancers/top-guru-assistants', 'https://www.guru.com/freelancers/eden-programming-solutions', 'https://www.guru.com/freelancers/avion-technology-inc-chicago', 'https://www.guru.com/freelancers/openwork-tech', 'https://www.guru.com/freelancers/j-consulting', 'https://www.guru.com/freelancers/smart-software-solutions-inc-482125', 'https://www.guru.com/freelancers/grata-tech-solutions', 'https://www.guru.com/freelancers/beksconsulting', 'https://www.guru.com/freelancers/stephenwhite', 'https://www.guru.com/freelancers/all-star-programmers', 'https://www.guru.com/freelancers/sakthiguru', 'https://www.guru.com/freelancers/illusion-software-development', 'https://www.guru.com/freelancers/robert-prislupsky', 'https://www.guru.com/freelancers/jalacrity', 'https://www.guru.com/freelancers/mv-squared', 'https://www.guru.com/freelancers/marco-samson', 'https://www.guru.com/freelancers/nathanial-mckay', 'https://www.guru.com/freelancers/joel-kim-1', 'https://www.guru.com/freelancers/taplabs']

activity = []
for i, val in enumerate(urls):
    html = val
    
    soup_html = html_extract()
    try:
        activity.append([(html, extract_30day_activity2())])
    except:a
        activity.append([(html, "NA")])

In [10]:
activity = [list(itertools.chain(*sub)) for sub in activity]
activity_df = pd.DataFrame(activity, columns = ['profile_url', 'quotes_30days'])

In [11]:
activity_df.head()

,profile_url,quotes_30days
0,https://www.guru.com/freelancers/scopic,NA
1,https://www.guru.com/freelancers/top-guru-assi...,NA
2,https://www.guru.com/freelancers/eden-programm...,NA
3,https://www.guru.com/freelancers/avion-technol...,NA
4,https://www.guru.com/freelancers/openwork-tech,NA


# Merging into Data

In [8]:
# Importing Data
sql_query = """SELECT * FROM user_details_table;"""
user_details = pd.read_sql_query(sql_query, con)

# Prepping
user_details['profile_url'] = "https://www.guru.com"+user_details['profile_url']

# Merging
dt = pd.merge(user_details, activity_df, on='profile_url')
dt = dt.drop(['index'], axis = 1)

# Saving back out
engine = create_engine('postgresql://%s:%s@localhost/%s' %
                       (username, pswd, dbname))
dt.to_sql("user_activity_details", engine, if_exists='replace')